# Quickstart

In this quickstart you will create a simple Llama Index App and learn how to log it and get feedback on an LLM response.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/frameworks/llama_index/llama_index_groundtruth.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
! pip install trulens-eval llama_index==0.8.3

### Add API keys
For this quickstart, you will need Open AI and Huggingface keys

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Import from LlamaIndex and TruLens

In [ ]:
# Imports main tools:
from trulens_eval import TruLlama, Feedback, Tru, feedback, FeedbackMode
from trulens_eval.feedback import GroundTruthAgreement
from trulens_eval.util import jsonify
tru = Tru()

### Create Simple LLM Application

This example uses LlamaIndex which internally uses an OpenAI LLM.

In [ ]:
# LLama Index starter example from: https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html
# In order to run this, download into data/ Paul Graham's Essay 'What I Worked On' from https://github.com/jerryjliu/llama_index/blob/main/examples/paul_graham_essay/data/paul_graham_essay.txt 

from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

### Send your first request

In [ ]:
#response = query_engine.query("What did the author do growing up?")
#print(response)

## Initialize Feedback Function(s)

In [ ]:
import numpy as np

# Initialize Huggingface-based feedback function collection class:
hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will check language match on the main app input and main app
# output.

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.min)

In [ ]:
golden_set = [
    {"query": "What was the author's undergraduate major?", "response": "He didn't choose a major, and customized his courses."},
    {"query": "What company did the author start in 1995?", "response": "Viaweb, to make software for building online stores."},
    {"query": "Where did the author move in 1998 after selling Viaweb?", "response": "California, after Yahoo acquired Viaweb."},
    {"query": "What did the author do after leaving Yahoo in 1999?", "response": "He focused on painting and tried to improve his art skills."},
    {"query": "What program did the author start with Jessica Livingston in 2005?", "response": "Y Combinator, to provide seed funding for startups."}
]

In [ ]:
f_groundtruth = Feedback(GroundTruthAgreement(golden_set).agreement_measure).on_input_output()

## Instrument chain for logging with TruLens

In [ ]:
tru_query_engine = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_lang_match, f_qa_relevance, f_qs_relevance, f_groundtruth],
    # feedback_mode=FeedbackMode.DEFERRED # should cause failure but does not
)

In [ ]:
# Instrumented query engine can operate like the original:
llm_response = tru_query_engine.query("Where was the author born?")

print(llm_response)

In [ ]:
#Run and evaluate on groundtruth questions
for pair in golden_set:
    llm_response = tru_query_engine.query(pair['query'])
    print(llm_response)
    

## Explore in a Dashboard

In [ ]:
# tru.run_dashboard() # open a local streamlit app to explore

# If running from github repo:
tru.run_dashboard(force=True, _dev=Path().cwd().parent.parent.parent.resolve()) # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all